In [16]:
import tensorflow as tf
import numpy      as np

SEQ_LEN    = 10
BATCH_SIZE = 5
N_OUTPUTS = 2  # in each sequence, 1-8 are features, and 9-10 is label
N_INPUTS = SEQ_LEN - N_OUTPUTS
DEFAULTS = [[0.0] for x in range(0, SEQ_LEN)]
TIMESERIES_COL = 'rawdata'

In [17]:
# read data and convert to needed format
def read_dataset(filename, mode=tf.contrib.learn.ModeKeys.TRAIN):
    def _input_fn():
        num_epochs = 2
        input_file_names = tf.train.match_filenames_once(filename)
    
        filename_queue = tf.train.string_input_producer(
            input_file_names, num_epochs=num_epochs, shuffle=True)
        reader = tf.TextLineReader()
        _, value = reader.read_up_to(filename_queue, num_records=BATCH_SIZE)

        value_column = tf.expand_dims(value, -1)
    
        # all_data is a list of tensors
        all_data = tf.decode_csv(value_column, record_defaults=DEFAULTS)  
        inputs = all_data[:len(all_data)-N_OUTPUTS]  # first few values
        label = all_data[len(all_data)-N_OUTPUTS : ] # last few values

        # from list of tensors to tensor with one more dimension
        inputs = tf.concat(inputs, axis=1)
        label = tf.concat(label, axis=1)
        return {TIMESERIES_COL: inputs}, label   # dict of features, label
    return _input_fn

In [18]:
# "inp" will have shape [BATCH_SIZE,NUM_INPUTS] where 
fn = read_dataset('train.csv')
data = fn()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    tf.train.start_queue_runners(coord=coord)
    try:
        while not coord.should_stop():
            inp, lab = sess.run(data)
            break
    except tf.errors.OutOfRangeError:
        print('All done')
    finally:
        coord.request_stop()

InvalidArgumentError: Number of ways to split should evenly divide the split dimension, but got split_dim 0 (size = 5) and num_split 8
	 [[Node: split_1 = Split[T=DT_FLOAT, num_split=8, _device="/job:localhost/replica:0/task:0/device:GPU:0"](split_1/split_dim, concat_4)]]
	 [[Node: split_1/_59 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_43_split_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'split_1', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/tom/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/tom/.local/lib/python3.5/site-packages/traitlets

In [ ]:
inp[1]